<a href="https://colab.research.google.com/github/bbberylll/ESAA_OB/blob/main/ESAA_OB_F2%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 핸즈온 7장 앙상블 학습과 랜덤 포레스트 p.245-257 필사  7.1~7.4

    화요일 : 7.1 ~ 7.2 수요일 7.3~7.4

2. 수상작 리뷰
  목요일 진행

## 앙상블 학습 & 랜덤 포레스트

### 앙상블
: 일련의 예측기(앙상블)로부터 예측을 수집하는 것

  * 최종 결과 선택은 분류-- 다수결, 회귀 -- 평균 계산

  * **앙상블 결과는 편향과 분산이 모두 감소. 주로 분산이 감소하는 효과가 유효함**

  1. 여러 개의 분류기를 훈련시킴 -> 직접 투표 분류기(hard voting - 다수결)

  2. 여러 개의 분류기를 훈련시킴 -> 간접 투표 (soft voting - 개별 분류기의 예측을 평균 낸다)

** **
  **높은 정확도를 보이려면 각기 다른 알고리즘으로 학습시켜 각 예측기가 서로 독립적으로 예측하도록 해야 함**

    --> 서로 독립적이고 오차에 상관관계가 없을 수록 높은 정확도 보임

    --> 이러한 가정은 같은 데이터로 훈련시키는 상황에는 맞지 않음. 따라서 **각기 다른 알고리즘으로 학습하는 것**이 중요


### 배깅과 페이스팅
Bagging = **Bootstrap aggregating**

= 훈련 세트에서 중복을 허용해 샘플링 하는 방식

** **

### oob 평가
oob = out of bag. 즉 선택되지 않은 훈련 샘플을 말함
이렇게 선택되지 않은 샘플을 이용해 validation을 진행할 수 있음!

이렇게 평가를 하면 accuracy_score를 이용한 값과 거의 유사한 정확도를 얻을 수 있게 된다.

** **

### 랜덤 패치와 랜덤 서브스페이스
* **이미지와 같은 고차원의 dataset에 유용**

* **더 다양한 예측기를 만들어 편향을 늘리지만 분산을 낮춤**
샘플링과 관련한 하이퍼 파라미터 : max_features, bootstrap_features

max_samples와 bootstrap와 유사하지만 샘플이 아닌 feature에 관한 샘플링임.

1. 랜덤 패치 방식 = feature와 sample을 모두 샘플링 하는 것
2. 랜덤 서브스페이스 방식 = 훈련 sample은 모두 사용하고, 특성은 sampling 하는 것  

** **

### 랜덤 포레스트
랜덤 포레스트 = 배깅 방법을 적용한 결정 트리의 앙상블임

랜덤 포레스트의 하이퍼 파라미터 = DecisionTree & Bagging 모두의 하이퍼 파라미터를 가지고 있음

전체 feature 중에서 적절한 feature을 찾기보다는
선택한 feature 중에서 최적의 feature을 찾는 방식으로 randomness를 주입

### 엑스트라 트리
= 극단적으로 무작위한 트리의 랜덤 포레스트 = 익스트림 랜덤 트리 앙상블 = 엑스트라 트리

* **편향을 늘어나지만 분산은 낮아진다**
* **일반적인 랜덤 포레스트보다 속도가 빠르다**

### 특성 중요도
랜덤 포레스트의 장점으로 **특성의 상대적 중요도를 측정하기 쉽다**

In [5]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y = make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [8]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8
RandomForestClassifier 0.95
SVC 0.95
VotingClassifier 0.95


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability=True)

voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft'
)

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()),
                             ('svc', SVC(probability=True))],
                 voting='soft')

In [15]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_proba = clf.predict_proba(X_test)
  y_pred = y_proba.argmax(axis=1)    ## 전체 배열에서 가장 높은 값을 가진 요소의 인덱스 배열
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8
RandomForestClassifier 0.95
SVC 0.95
VotingClassifier 0.95


In [17]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=50, bootstrap=True, n_jobs=-1    # n_jobs = 사용할 CPU 코어 갯수 설정하는 hyper-parameter
)                                                 # -1로 지정하면 가용한 모든 코어를 사용함

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [20]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True
)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9625

In [21]:
from sklearn.metrics import accuracy_score

y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.95

In [22]:
bag_clf.oob_decision_function_   ## oob 샘플이 각 class에 속할 확률을 계산한 결정 함수 값을 확인할 수 있음

array([[0.44623656, 0.55376344],
       [1.        , 0.        ],
       [0.99401198, 0.00598802],
       [1.        , 0.        ],
       [0.91099476, 0.08900524],
       [0.00520833, 0.99479167],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.79532164, 0.20467836],
       [0.28342246, 0.71657754],
       [0.74860335, 0.25139665],
       [0.29281768, 0.70718232],
       [0.        , 1.        ],
       [0.50819672, 0.49180328],
       [0.        , 1.        ],
       [0.89385475, 0.10614525],
       [0.        , 1.        ],
       [0.89944134, 0.10055866],
       [0.00549451, 0.99450549],
       [0.20634921, 0.79365079],
       [0.96808511, 0.03191489],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.05291005, 0.94708995],
       [1.        , 0.        ],
       [0.98895028, 0.01104972],
       [1.        , 0.        ],
       [0.02941176, 0.97058824],
       [0.

In [23]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred = rnd_clf.predict(X_test)

In [24]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

In [25]:
from sklearn.datasets import load_iris

iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])

for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.10908598165231055
sepal width (cm) 0.022217870442766168
petal length (cm) 0.45304398805202994
petal width (cm) 0.4156521598528934
